In [ ]:
import os
import random
import pandas as pd
from imageio import imread
import math
import numpy as np
import cv2
import keras
import seaborn as sns
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential
import matplotlib.pyplot as plt

In [ ]:
data_dir = os.path.abspath('../input/gtsrb-german-traffic-sign')
os.path.exists(data_dir)

In [ ]:
def resize_cv(im):
    return cv2.resize(im, (64, 64), interpolation = cv2.INTER_LINEAR)

In [ ]:
#train_file = pd.read_csv(data_dir + "/Train.csv")
#for index, row in train_file.iterrows():

In [ ]:
list_images = []
output = []
train_file = pd.read_csv(data_dir + "/Train.csv")
for index, row in train_file.iterrows():
    img_path = data_dir + "/" +  row['Path']
    img=imread(img_path)
    img = img[row['Roi.X1']:row['Roi.X2'],row['Roi.Y1']:row['Roi.Y2'],:]
    img = resize_cv(img)
    list_images.append(img)
    output.append(row.ClassId)

fig = sns.distplot(output, kde=False, bins = 43, hist = True, hist_kws=dict(edgecolor="black", linewidth=2))
fig.set(title = "Traffic signs frequency graph",
        xlabel = "ClassId",
        ylabel = "Frequency")

In [ ]:
example_im=[]
for i in range(3):
    random_filename = random.choice([
        x for x in list_images
    ])
    example_im.append(random_filename)
    
fig = plt.figure(figsize=(9, 13))
ax=[]
for i in range(3):
    ax.append( fig.add_subplot(1, 3, i+1) )
    plt.imshow(example_im[i])
    
ax[1].set_title("Sample Images")
plt.show() 

In [ ]:
input_x = np.stack(list_images)
input_y = keras.utils.to_categorical(output)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) 


In [ ]:
hidden_num_units = 2048
hidden_num_units1 = 1024
hidden_num_units2 = 128
output_num_units = 43
epochs = 10
batch_size = 16
pool_size = (2, 2)
#list_images /= 255.0
input_shape = Input(shape=(32, 32,3))
model = Sequential([
Conv2D(16, (3, 3), activation='relu', input_shape=(64,64,3), padding='same'),
 BatchNormalization(),
Conv2D(16, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
 MaxPooling2D(pool_size=pool_size),
 Dropout(0.2),
    
 Conv2D(32, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
    
 Conv2D(32, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
 MaxPooling2D(pool_size=pool_size),
 Dropout(0.2),
    
 Conv2D(64, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
    
 Conv2D(64, (3, 3), activation='relu', padding='same'),
 BatchNormalization(),
 MaxPooling2D(pool_size=pool_size),
 Dropout(0.2),
Flatten(),
Dense(units=hidden_num_units, activation='relu'),
 Dropout(0.3),
 Dense(units=hidden_num_units1, activation='relu'),
 Dropout(0.3),
 Dense(units=hidden_num_units2, activation='relu'),
 Dropout(0.3),
 Dense(units=output_num_units, input_dim=hidden_num_units, activation='softmax'),
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
trained_model_conv = model.fit(X_train.reshape(-1,64,64,3), y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save('my_model.h5')

In [ ]:
model = keras.models.load_model('my_model.h5')

In [ ]:
model = keras.models.load_model('my_model.h5')
list_images_test = []
output_test = []
test_file = pd.read_csv(data_dir + "/Test.csv")
for index, row in train_file.iterrows():
    img_path = data_dir + "/" +  row['Path']
    img=imread(img_path)
    img = img[row['Roi.X1']:row['Roi.X2'],row['Roi.Y1']:row['Roi.Y2'],:]
    img = resize_cv(img)
    list_images_test.append(img)
    output_test.append(row.ClassId)
    
pred_x = np.stack(list_images_test)
pred_y = keras.utils.to_categorical(output_test)
model.evaluate(pred_x, pred_y)



In [ ]:
names= pd.read_csv('../input/sign-names/Sign names.csv',header=None)

In [ ]:
model = keras.models.load_model('my_model.h5')
example_pred=[]
for i in range(3):
    random_filename = random.choice([
        x for x in pred_x
    ])
    example_pred.append(random_filename)
    
fig2 = plt.figure(figsize=(9, 30))
ax2=[]
for i in range(3):
    ax2.append( fig2.add_subplot(1, 3, i+1) )
    plt.imshow(example_pred[i])

example_pred = np.array(example_pred)
predictions = model.predict(example_pred)
classes = np.argmax(predictions, axis = 1)
ax2[0].set_xlabel(names[0][classes[0]])
ax2[1].set_xlabel(names[0][classes[1]])
ax2[2].set_xlabel(names[0][classes[2]])
plt.tight_layout()
plt.show()


In [ ]:
print(type(pred_x))